<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/master/tutorials_W0D4_Calculus_W0D4_Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromatch Academy: Week 0, Day 4, Tutorial 3
# Numerical Methods

__Content creators:__ John S Butler, Arvind Kumar

__Content reviewers:__ Swapnil Kumar

__Production editors:__ Matthew McCann, Dimuthu Hemachandra


---
# Tutorial Objectives

While a great deal of neuroscience can be describe by mathematics a great deal of the mathematics used cannot be solved exactly. This might seem very odd that you can writing something in mathematics that cannot be immediately solved but that is the beauty and mystery of mathematics. To side step this issue we use Numerical Methods to estimate the solution.



In this tutorial, we will look at the Euler method to estimate the solution of a few different differential equations, the population equation, the Leaky Integrate and Fire model and a simplified version of the Wilson-Cowan model which is a system of differential equations.

**Steps:**
- Code the Euler estimate of the Population Equation;
- Investigate the impact of time step on the error of the numerical solution;
- Code the Euler estimate of the Learky Integrate and Fire model for a constant input;
- Visualize and listen to the response of the integrate for different inputs;
- Apply the Euler method to estimate the soluton of a system of differential equations.



In [ ]:
#@markdown Tutorial slides
# you should link the slides for all tutorial videos here (we will store pdfs on osf)

from IPython.display import HTML
HTML('<iframe src="" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# @title Figure Settings
import IPython.display as ipd
from matplotlib import gridspec

import ipywidgets as widgets  # interactive display
from ipywidgets import Label
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()


In [ ]:
# @title Plotting Functions

time = np.arange(0, 1, 0.01)

def plot_slope(dt):
  """
    Args:
      dt  : time-step
    Returns:
      A figure of an exponential, the slope of the exponential and the derivative exponential
  """

  t = np.arange(0, 5+0.1/2, 0.1)

  with plt.xkcd():

    fig = plt.figure(figsize=(6, 4))
    # Exponential
    p = np.exp(0.3*t)
    plt.plot(t, p, label='y')
    # slope
    plt.plot([1, 1+dt], [np.exp(0.3*1), np.exp(0.3*(1+dt))],':og',label=r'$\frac{y(1+\Delta t)-y(1)}{\Delta t}$')
    # derivative
    plt.plot([1, 1+dt], [np.exp(0.3*1), np.exp(0.3*(1))+dt*0.3*np.exp(0.3*(1))],'-k',label=r'$\frac{dy}{dt}$')
    plt.legend()
    plt.plot(1+dt, np.exp(0.3*(1+dt)), 'og')
    plt.ylabel('y')
    plt.xlabel('t')
    plt.show()



def plot_StepEuler(dt):
  """
    Args:
      dt  : time-step
    Returns:
      A figure of one step of the Euler method for an exponential growth function
  """

  t=np.arange(0, 1 + dt + 0.1 / 2, 0.1)

  with plt.xkcd():
    fig = plt.figure(figsize=(6,4))
    p=np.exp(0.3*t)
    plt.plot(t,p)
    plt.plot([1,],[np.exp(0.3*1)],'og',label='Known')
    plt.plot([1,1+dt],[np.exp(0.3*1),np.exp(0.3*(1))+dt*0.3*np.exp(0.3*1)],':g',label=r'Euler')
    plt.plot(1+dt,np.exp(0.3*(1))+dt*0.3*np.exp(0.3*1),'or',label=r'Estimate $p_1$')
    plt.plot(1+dt,p[-1],'bo',label=r'Exact $p(t_1)$')
    plt.vlines(1+dt,np.exp(0.3*(1))+dt*0.3*np.exp(0.3*1),p[-1],colors='r', linestyles='dashed',label=r'Error $e$')
    plt.legend()
    plt.ylabel('Population (millions)')
    plt.xlabel('time(years)')
    plt.show()

## LIF PLOT
def plot_IF(t, V, I, Spike_time):
  """
    Args:
      t  : time
      V  : membrane Voltage
      I  : Input
      Spike_time : Spike_times
    Returns:
      figure with three panels
      top panel: Input as a function of time
      middle panel: membrane potential as a function of time
      bottom panel: Raster plot
  """

  with plt.xkcd():
    fig = plt.figure(figsize=(12,4))
    gs = gridspec.GridSpec(3, 1,  height_ratios=[1, 4, 1])
    # PLOT OF INPUT
    plt.subplot(gs[0])
    plt.ylabel(r'$I_e(nA)$')
    plt.yticks(rotation=45)
    plt.plot(t,I,'g')
    #plt.ylim((2,4))
    plt.xlim((-50,1000))
    # PLOT OF ACTIVITY
    plt.subplot(gs[1])
    plt.plot(t,V,':')
    plt.xlim((-50,1000))
    plt.ylabel(r'$V(t)$(mV)')
    # PLOT OF SPIKES
    plt.subplot(gs[2])
    plt.ylabel(r'Spike')
    plt.yticks([])
    plt.scatter(Spike_time,1*np.ones(len(Spike_time)), color="grey", marker=".")
    plt.xlim((-50,1000))
    plt.xlabel('time(ms)')
    plt.show()

def plot_rErI(t, r_E, r_I):
  """
    Args:
      t   : time
      r_E : excitation rate
      r_I : inhibition rate

    Returns:
      figure of r_I and r_E as a function of time

  """
  with plt.xkcd():
    fig = plt.figure(figsize=(6,4))
    plt.plot(t,r_E,':',color='b',label=r'$r_E$')
    plt.plot(t,r_I,':',color='r',label=r'$r_I$')
    plt.xlabel('time(ms)')
    plt.legend()
    plt.ylabel('Firing Rate (Hz)')
    plt.show()


def plot_rErI_Simple(t, r_E, r_I):
  """
    Args:
      t   : time
      r_E : excitation rate
      r_I : inhibition rate

    Returns:
      figure with two panels
      left panel: r_I and r_E as a function of time
      right panel: r_I as a function of r_E with Nullclines

  """
  with plt.xkcd():
    fig = plt.figure(figsize=(12,4))
    gs = gridspec.GridSpec(1, 2)
    # LEFT PANEL
    plt.subplot(gs[0])
    plt.plot(t,r_E,':',color='b',label=r'$r_E$')
    plt.plot(t,r_I,':',color='r',label=r'$r_I$')
    plt.xlabel('time(ms)')
    plt.legend()
    plt.ylabel('Firing Rate (Hz)')
    # RIGHT PANEL
    plt.subplot(gs[1])
    plt.plot(r_E,r_I,'k:')
    plt.plot(r_E[0],r_I[0],'go')

    plt.hlines(0,np.min(r_E),np.max(r_E),linestyles="dashed",color='b',label=r'$\frac{d}{dt}r_E=0$')
    plt.vlines(0,np.min(r_I),np.max(r_I),linestyles="dashed",color='r',label=r'$\frac{d}{dt}r_I=0$')

    plt.legend(loc='upper left')

    plt.xlabel(r'$r_E$')
    plt.ylabel(r'$r_I$')
    plt.show()

def plot_rErI_Matrix(t, r_E, r_I, Null_rE, Null_rI):
  """
    Args:
      t   : time
      r_E : excitation firing rate
      r_I : inhibition firing rate
      Null_rE: Nullclines excitation firing rate
      Null_rI: Nullclines inhibition firing rate
    Returns:
      figure with two panels
      left panel: r_I and r_E as a function of time
      right panel: r_I as a function of r_E with Nullclines

  """

  with plt.xkcd():
    fig = plt.figure(figsize=(12,4))
    gs = gridspec.GridSpec(1, 2)
    plt.subplot(gs[0])
    plt.plot(t,r_E,':',color='b',label=r'$r_E$')
    plt.plot(t,r_I,':',color='r',label=r'$r_I$')
    plt.xlabel('time(ms)')
    plt.ylabel('Firing Rate (Hz)')
    plt.legend()
    plt.subplot(gs[1])
    plt.plot(r_E,r_I,'k:')
    plt.plot(r_E[0],r_I[0],'go')

    plt.plot(r_E,Null_rE,':',color='b',label=r'$\frac{d}{dt}r_E=0$')
    plt.plot(r_E,Null_rI,':',color='r',label=r'$\frac{d}{dt}r_I=0$')
    plt.legend(loc='best')
    plt.xlabel(r'$r_E$')
    plt.ylabel(r'$r_I$')
    plt.show()

--- 
# Section 1 Euler Method

In [ ]:
#@title Video 1: Intro to Numerical Methods and the Euler Method
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="ORbqe7TamVI", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

The Euler method is one of the most simple and elegant methods to approximate a differential. It was designed by [Leonhard Euler](https://en.wikipedia.org/wiki/Leonhard_Euler) (1707-1783). 
Simply put we just replace the derivative in the differential equation by the formula for a line and re-arrange.



## Section 1.1: Slope of a line
The slope is the rate of change between two points. The formula for the slope of a line between the points $(t_0,y(t_0))$ and $(t_1,y(t_1))$ is given by:
$$ m=\frac{y(t_1)-y(t_0)}{t_1-t_0}, $$
which can be written as 
$$ m=\frac{y_1-y_0}{t_1-t_0}, $$
or as
$$ m=\frac{\Delta y_0}{\Delta t_0}, $$
where $\Delta y_0=y_1-y_0$ and $\Delta t_0=t_1-t_0$ or in words as
$$ m=\frac{\text{ Change in y} }{\text{Change in t}}. $$
The slope can be used an approximation of the derivative such that
$$ \frac{d}{dt}y(t)\approx \frac{y(t_0+\Delta t)-y(t_0)}{t_0+\Delta t-t_0}=\frac{y(t_0+dt)-y(t_0)}{\Delta t}$$
where $\Delta t$ is a time-step.

### Interactive Demo 1.1: Slope of a Line
The plot below shows a function $y(t)$ in blue, its exact derivative $ \frac{d}{dt}y(t)$ at $t_0=1$ in black and the approximate derivative calculated using the slope formula $=\frac{y(1+\Delta t)-y(1)}{\Delta t}$ for different time-steps sizes $\Delta t$ in green.

Interact with the widget to see how time-step impacts the accuracy of the slope which is the estimate of the derivative.

In [ ]:
# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(

    dt=widgets.FloatSlider(1, min=0., max=4., step=.1,
                                layout=my_layout)

)

def Pop_widget(dt):
  plot_slope(dt)
  plt.show()

### Think! 1.1: How does the size of $dt$ affect the approximation?


[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_a4085ae8.py)



---
# Section 2: Population Differential Equation

Linking with the differential equation tutorial, we will use the population to get an intuitive feel of Euler method to approximate the solution of a differential equation. We start with the population because it has an exact solution with no discontinuities unlike the Leaky Integrate and Fire model.

The population differential equation is
\begin{align*}
\\
\frac{d}{dt}\,p(t) &= \alpha p(t)\\\\
p(0)&=p_0 \quad \text{Initial Condition}
\end{align*}

where $p(t)$ is the population of the world and $\alpha$ is a parameter representing birth rate, with
the exact solution 
$$ p(t)=p_0e^{\alpha}.$$

To numerically approximate the population differential equation we replace the derivate with the slope of the line to get the discrete (not continuous) equation:

\begin{align*}
\\
\frac{p_1-p_0}{t_1-t_0} &= \alpha p_0\\\\
p(0)&=p_0 \quad \text{Initial Condition}
\end{align*}

where $p_1$ is the approximate of $p(t_1)$. Let $\Delta t=t_1-t_0$ be the time step and re-arrange the equation gives

\begin{align*}
\color{red}{p_1}&=\color{green}{p_0}+\color{blue}{\Delta t} (\color{blue}{\alpha} \color{green}{p_0})
\end{align*}

where $\color{red}{p_1}$ is the unknown future, $\color{green}{p_0}$ is the known current population, $\color{blue}{\Delta t}$ is the chosen time-step parameter and $\color{blue}{\alpha}$ is the given birth rate parameter.
Another way to read the re-agranged discrete equation is:

\begin{align*}
\color{red}{\text{"Future Population"}}&=\color{green}{\text{ "Current Population"}}+\color{blue}{\text{ time-step}} \times (\color{blue}{\text{ "Birth rate}}\text{ times } \color{green}{\text{ Current Population"}}).
\end{align*}

So pretty much we can use the current population and add a bit of the dynamics of the differential equation to predict the future. We will make millions, but wait there is always an error.

## Exercise 2: One step of Euler Method
Given the population differential equation
\begin{align*}
\\
\frac{d}{dt}\,p(t) &= 0.3 p(t),\\\\
p(t_0=1)&=e^{0.3},\quad \text{Initial Condition}
\end{align*}

code the Euler method to estimate the population $p_1$ at $t_1=2$ given the time-step $\Delta t=1$ (for brevity, $\Delta t$ is `dt` in the code).

In [ ]:
dt = 1    # time-step
t_0 = 1   # initial time
p_0 = np.exp(0.3*t_0)    # initial population

########################################################################
## TODO for students: calculate the estimate solution of p at t_1
## Complete line of code and remove
raise NotImplementedError("Student exercise: calculate the estimate solution of p at t_1")
########################################################################

dp = ...
p_1 = ...
print(p_1)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_a7d8b707.py)



---
# Section 3: Euler Error

The solution of the Euler method $p_1$ is an estimate of the exact solution $p(t_1)$ at $t_1$ which means there is a bit of error $e_1$ which gives the equation
\begin{align*}
p(t_1)&=p_1+e_1\\\\
\text{Rearranging}\\\\
e_1&=p(t_1)-p_1.
\end{align*}

Most of the time we do not know the exact answer $p(t_1)$ but in the population equation we have the exact solution

$$ p(t)=p_0e^{\alpha}.$$

This means we can explore what the error looks like.


## Interactive Demo 3: Euler Error
Interact with the widget to see how the time-step $dt$ impacts the estimate $p_1$ and the error $e_1$ of the Euler method.

In [ ]:
# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    dt=widgets.FloatSlider(.3, min=0., max=4., step=.1,
                                layout=my_layout)
)

def Pop_widget(dt):
  plot_StepEuler(dt)
  plt.show()

## Think! 3: Effect of $\Delta t$ on Euler Error

1. What happens to the estimate $p_1$ as the time-step $\Delta t$ increases?

2. Is there a relationship between the size of $\Delta t$ and $e_1$?

3. How would you improve the error $e_1$?

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_6376082e.py)



The error $e_1$ from one timestep is known as the __local error__. For the Euler method the local error is linear, that means that the error decreases linearly as a function of timestep $O(\Delta t)$.

## Section 3.1: Time-Steps
In the above exercise we saw that by increasing the step size $dt$ the error between the estimate $p_1$ and the exact $p(t_1)$ increased with the largest error in the example above was for $\Delta t=4$ which gave the interval $[1, 5]$,
$$n=\frac{5-1}{\Delta t}=\frac{5-1}{4}=1,$$
hence only $1$ time-step of size $dt=4$, giving
$$ t_0=1,\ \text{ and } t_1=5. $$

To address decrease the error we divide the interval $[1, 5]$ into more steps using a smaller $\Delta t$. 
In the plot below we use $\Delta t=1$, which divides the interval into four segments
$$n=\frac{5-1}{1}=4,$$
giving
$$ t_0=1, \ t_1=2, \ t_2=3, \ t_3=4 \ \text{ and } t_4=5. $$
This can be written as
$$ t_i=1+i\times1, \text{ for } i=0,\cdots 4, $$
and more generally as 
$$ t_i=t_0+i\times dt, \text{ for } i=0,\cdots n. $$

In [ ]:
# @markdown *Execute this cell to visualize time steps*
dt  =1
t  =np.arange(1, 5+dt/2, dt) # Time from 1 to 5 years in 0.1 steps
with plt.xkcd():
    fig = plt.figure(figsize=(8, 2))
    plt.plot(t, 0*t, ':o')
    plt.plot(t[0] ,0*t[0],':go',label='Initial Condition')
    plt.text(t[0]-0.1, 0*t[0]-0.03, r'$t_0$')
    plt.text(t[1]-0.1, 0*t[0]-0.03, r'$t_1$')
    plt.text(t[2]-0.1, 0*t[0]-0.03, r'$t_2$')
    plt.text(t[3]-0.1, 0*t[0]-0.03, r'$t_3$')
    plt.text(t[4]-0.1, 0*t[0]-0.03,r'$t_4$')
    plt.text(t[0]+0.5, 0*t[0]+0.02, r'$\Delta t$')
    plt.text(t[1]+0.5, 0*t[0]+0.02, r'$\Delta t$')
    plt.text(t[2]+0.5, 0*t[0]+0.02, r'$\Delta t$')
    plt.text(t[3]+0.5, 0*t[0]+0.02, r'$\Delta t$')
    plt.yticks([])#plt.ylabel('Population (millions)')
    plt.xlabel('time(years)')
    plt.show()

## Section 3.2: The Difference Equation
Using the Euler method the differential equation is written as a series of difference equations of the form:
\begin{align*}
\color{red}{p_{i+1}}&=\color{green}{p_i}+\color{blue}{\Delta t} (\color{blue}{\alpha} \color{green}{p_i})\\
&\text{for } i=0,1,\cdots n-1
\end{align*}
where $\color{red}{p_{i+1}}$ is the unknown estimate of the future population at $t_{i+1}$, $\color{green}{p_i}$ is the known current population estimate at $t_i$, $\color{blue}{\Delta t}$ is the chosen time-step parameter and $\color{blue}{\alpha}$ is the given birth rate parameter.

### Coding Exercise 3.2: Step, step, step
Given the population differential equation
\begin{align*}
\\
\frac{d}{dt}\,p(t) &= 0.3 p(t),\\
p(t_0=1)&=e^{0.3},\quad \text{Initial Condition,}
\end{align*}

code the difference equation

\begin{align*}
\color{red}{p_{i+1}}&=\color{green}{p_i}+\color{blue}{dt} (\color{blue}{0.3} \color{green}{p_i}),\\
\color{green}{p_0}&=e^{0.3},\quad \text{Initial Condition,}\\
&\text{for } i=0,1,\cdots 4,\\
\end{align*}

to estimate the population on the interval $[1,5]$ with a time-step $dt=1$.

In [ ]:
dt = 1
t = np.arange(1, 5+dt/2, dt) # Time from 0 to 10 years in 0.1 steps
n = len(t)
p = np.ones(n)
p[0] = np.exp(0.3*t[0]) # Initial Condition

fig = plt.figure(figsize=(6, 4))
plt.plot(t, np.exp(0.3*t), 'k', label='Exact Solution')

for i in range(n-1):
  ########################################################################
  ## TODO for students: calculate the population step for each time point
  ## Complete line of code and remove
  raise NotImplementedError("Student exercise: calculate the population step for each time point")
  ########################################################################

  dp = ...
  p[i+1] = ...

plt.plot(t, p,':o', label='Euler Estimate')
plt.vlines(t, p, np.exp(0.3*t),
           colors='r', linestyles='dashed', label=r'Error $e_i$')

plt.ylabel('Population (millions)')
plt.legend()
plt.xlabel('Time (years)')
plt.show()

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_92263c87.py)

*Example output:*

<img alt='Solution hint' align='left' width=416 height=272 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials//static/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_92263c87_0.png>



The error is smaller for 4 time-step than taking one large time step from 1 to 5 but do note that the error is increasing for each step, this is known as __global error__ so the futher in time you want to predicted the larger the error.
You can read the theorems [here.](https://colab.research.google.com/github/john-s-butler-dit/Numerical-Analysis-Python/blob/master/Chapter%2001%20-%20Euler%20Methods/102_Euler_method_with_Theorems_nonlinear_Growth_function.ipynb) 

---
# Section 4: Generalised Euler Method

The Euler method can be applied to all first order differential equations of the form
\begin{align*}
\frac{d}{dt}y(t)&=f(t,y(t)),\\
y(t_{0})&=y_0,\\
\end{align*}
on an interval $[a,b]$.

The differential equation can be written using the Euler method as
\begin{align*}
\frac{\color{red}{y_{i+1}}-\color{green}{y_i}}{\color{blue}{\Delta t}}&=f(\color{blue}{t_i},\color{green}{y_i}),\\
\color{red}{y_{i+1}}&=\color{green}{y_i}+\color{blue}{\Delta t}f(\color{blue}{t_i},\color{green}{y_i}),\\
&\text{ for } i=0, \cdots n-1,\\
y(\color{blue}{t_{0}})&=\color{green}{y_0},\\
\end{align*}
where  $\color{red}{y_{i+1}}$ is the unknown estimate at $t_{i+1}$, $\color{green}{y_i}$ is the known at $t_i$, $\color{blue}{\Delta t}$ is the chosen time-step parameter, $\color{blue}{t_i}$ is the time point and $f$ is the right handside of the differential equation. 
The time discrete time stesp are:
\begin{align*}
\color{blue}{t_{i}}&=\color{blue}{t_0}+\color{blue}{i}\color{blue}{\Delta t},\\
n&=\frac{b-a}{\Delta t}\\
&\text{ for } i=0, \cdots n.\\
\end{align*}
Once again this can be simply put into words:
\begin{align*}
\color{red}{\text{ "Future" }}&=\color{green}{\text{ "Current Info" }}+\color{blue}{\text{ time-step } }\times\text{ "Dynamics of the system which is a function of } \color{blue}{ \text{ time }} \text{ and }\color{green}{ \text{ Current Info." }}
\end{align*}

---
# Section 5: Euler Method for the Leaky Integrate and Fire

In [ ]:
#@title Video 2: Leaky Integrate and Fire Model
video = YouTubeVideo(id="JWI2msDBZ7o", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

The Leaky Integrare and Fire differential equation is:

\begin{align}
\frac{dV}{dt} = \frac{-(V-E_L) + R_mI(t)}{\tau_m}\,
\end{align}

where $\tau_m$ is the time constant, $V$ is the membrane potential, $E_L$ is the resting potential, $R_m$ is leak resistance and $I(t)$ is the external input current. 

The solution can be estimated by applyting the Euler method to give the difference equation:

\begin{align}
\frac{\color{red}{V_{i+1}}-\color{green}{V_{i}}}{\color{blue}{\Delta t}}=\big(\frac{-(\color{green}{V_i}-\color{blue}{E_L}) + \color{blue}{R_m}I_i}{\color{blue}{\tau_m}}\big),\\
\end{align}

Re-arranging gives:

\begin{align}
\color{red}{V_{i+1}}=\color{green}{V_{i}}+\color{blue}{\Delta t}\big(\frac{-(\color{green}{V_i}-\color{blue}{E_L}) + \color{blue}{R_m}I_i}{\color{blue}{\tau_m}}\big),\\
\text{for } i=0\cdots n-1,
\end{align}

where $\color{red}{V_{i+1}}$ is unknown, $\color{green}{V_{i}} $ is known, $\color{blue}{E_L}$, $\color{blue}{R_m}$ and $\color{blue}{\tau_m}$ are known parameters, $\color{blue}{\Delta t}$ is a chosen time-step and  $I(t_i)$ is a function for an external input current.

## Coding Exercise 5: LIF and Euler
Code the difference equation for the LIF:

\begin{align}
\color{red}{V_{i+1}}=\color{green}{V_{i}}+\color{blue}{\Delta t}\big(\frac{-(\color{green}{V_i}-\color{blue}{E_L}) + \color{blue}{R_m}I_i}{\color{blue}{\tau_m}}\big),\\
\text{for } i=0\cdots n-1,
\end{align}

with the given parameters set as:
* `V_reset = -75,`
* `E_L = -75,`
* `tau_m = 10,`
* `R_m = 10.`


The plot below is a figure with with three panels; 
* the top panel is the sinusoidal input, $I$, 
* the middle panel is the estimate membrane potential $V_i$ to illustrative the spike $V_i$ is set to $0$ and then reset, 
* the bottom panel is the raster plot with each dot indicating a spike.

As electrophysiologists normally listen to spikes when conducting experiments,  listen to the music of different firing rates by changing the input value $I$. 

In [ ]:
def Euler_Integrate_and_Fire(I, time, dt):
  """
  Args:
    I: Input
    time: time
    dt: time-step
  Returns:
    Spike: Spike count
    Spike_time: Spike times
    V: membrane potential esitmated by the Euler method
  """

  Spike = 0
  tau_m = 10
  R_m = 10
  t_isi = 0
  V_reset = E_L = -75
  n = len(time)
  V = V_reset * np.ones(n)
  V_th = -50
  Spike_time = []

  for i in range(n-1):
    #######################################################################
    ## TODO for students: calculate the estimate solution of V at t[i+1]
    ## Complete line of codes for dV and remove
    ## Run the code in Section 5.1 or 5.2 to see the output!
    raise NotImplementedError("Student exercise: calculate the estimate solution of V at t[i+1]")
    ########################################################################

    dV = ...
    V[i+1] = V[i] + dt*dV

    # Discontinuity for Spike
    if V[i] > V_th:
      V[i] = 0
      V[i+1] = V_reset
      t_isi = time[i]
      Spike = Spike + 1
      Spike_time = np.append(Spike_time, time[i])

  return Spike, Spike_time, V

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_c27085de.py)



## Section 5.1: Sinusoidal Input to LIF
The plot below shows the estimated solution of the LIF using the Euler method for a sinusoidal input.

In [ ]:
dt = 1
t = np.arange(0, 1000, dt)
I = np.sin(4 * 2 * np.pi * t/1000) + 2
Spike, Spike_time, V = Euler_Integrate_and_Fire(I, t, dt)

plot_IF(t, V,I,Spike_time)
ipd.Audio(V, rate=1000/dt)
plt.show()

### Section 5.2: Random Input to the LIF
The plot below shows a figure with with three panels; 
* the top panel is the noisy input, $I_i$, 
* the middle panel is the estimate membrane potential $V_i$ to illustrative the spike $V_i$ is set to $0$ and then reset, 
* the bottom panel is the raster plot with each dot indicating a spike.

Listen to the music of different firing rates for a noisy input $I$. 

In [ ]:
dt = 1
t = np.arange(0, 1000, dt)
I = np.random.randn(len(t)) + 2.5
Spike, Spike_time, V = Euler_Integrate_and_Fire(I, t, dt)

plot_IF(t, V, I, Spike_time)
ipd.Audio(V, rate=1000/dt)
plt.show()

---
# Section 6: Systems of Differential Equations

In [ ]:
#@title Video 3: Systems of Differential Equations
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="0rTYV7g0edM", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

To get to grips with solving a system of differential equations using the Euler method, we will simplify the Wilson Cowan model, a set of equations that will be exploted in more detail on W?D?. The next few sections will apply the Euler method to a system of equations and will also give you a feel of a how to investigate a system of differential equations.

We will start with a linear version of the Wilson Cowan model. Consider the equations, 

\begin{align}
\tau_E \frac{dr_E}{dt} &= w_{EE}r_E+w_{EI}r_I \\
\tau_I \frac{dr_I}{dt} &= w_{IE}r_E+w_{II}r_I
\end{align}

$r_E(t)$ represents the average activation (or firing rate) of the excitatory population at time $t$, and $r_I(t)$ the activation (or firing rate) of the inhibitory population. The parameters $\tau_E$ and $\tau_I$ control the timescales of the dynamics of each population. Connection strengths are given by: $w_{EE}$ (E $\rightarrow$ E), $w_{EI}$ (I $\rightarrow$ E), $w_{IE}$ (E $\rightarrow$ I), and $w_{II}$ (I $\rightarrow$ I). The terms $w_{EI}$ and $w_{IE}$ represent connections from inhibitory to excitatory population and vice versa, respectively. 

## Coding Exercise 6: Euler on a Simple System 
We will further simplify the equations by setting $w_{EE}$ and $w_{II}$ to zero, we now have the equations 

\begin{align}
\tau_E \frac{dr_E}{dt} &= w_{EI}r_I, \\
\tau_I \frac{dr_I}{dt} &= w_{IE}r_E, \qquad (1)
\end{align}

where $\tau_E=100$ and $\tau_I=120$, no internal connection $w_{EE}=w_{II}=0$, and $w_{EI}=-1$ and $w_{IE}=1$,
with the initial conditions

\begin{align}
r_E(0)=30, \\ 
r_I(0)=20.
\end{align}

The solution can be approximated using the Euler method such that we have the difference equations:

\begin{align*}
\frac{\color{red}{rE_{i+1}}-\color{green}{rE_i}}{\color{blue}{\Delta t}}&=\big(\frac{\color{blue}{w_{EI}}\color{green}{rI_i}}{\color{blue}{\tau_E}}\big),\\
\frac{\color{red}{rI_{i+1}}-\color{green}{rI_i}}{\color{blue}{\Delta t}}&=\big(\frac{\color{blue}{w_{IE}}\color{green}{rE_i}}{\color{blue}{\tau_I}}\big),\\
\end{align*}

Re-arranging:

\begin{align*}
\color{red}{rE_{i+1}}&=\color{green}{rE_i}+\color{blue}{\Delta t}\big(\frac{\color{blue}{w_{EI}}\color{green}{rI_i}}{\color{blue}{\tau_E}}\big),\\
\color{red}{rI_{i+1}}&=\color{green}{rI_i}+\color{blue}{\Delta t}\big(\frac{\color{blue}{w_{IE}}\color{green}{rE_i}}{\color{blue}{\tau_I}}\big),\\
&\text{ for  } i=0, \cdots , n-1,\\\\
rE_0&=30,\\
rI_0&=20.\\
\end{align*}

where $\color{red}{rE_{i+1}}$ and  $\color{red}{rI_{i+1}}$ are unknown, $\color{green}{rE_{i}} $ and  $\color{green}{rI_{i}} $ are known, $\color{blue}{w_{EI}}=-1$, $\color{blue}{w_{IE}}=1$ and $\color{blue}{\tau_E}=100$ and $\color{blue}{\tau_I}=120$  are known parameters and $\color{blue}{\Delta t}$ is a chosen time-step.

__Code the difference equations to estimate $r_{E}$ and $r_{I}$.__

Note that the equations have to estimated in the same `for` loop as they depend on each other.

In [ ]:
def Euler_Simple_Linear_System(t, dt):
  """
  Args:
    time: time
  dt  : time-step
  Returns:
    r_E : Excitation Firing Rate
    r_I : Inhibition Firing Rate

  """

  tau_E = 100
  tau_I = 120
  n = len(t)
  r_I = 20*np.ones(n)
  r_E = 30*np.ones(n)

  #######################################################################
  ## TODO for students: calculate the estimate solutions of r_E and r_I at t[i+1]
  ## Complete line of codes for dr_E and dr_I and remove
  raise NotImplementedError("Student exercise: calculate the estimate solutions of r_E and r_I at t[i+1]")
  ########################################################################

  for i in range(n-1):
    dr_E = ...
    r_E[i+1] = r_E[i] + dt*dr_E

    dr_I = ...
    r_I[i+1] = r_I[i] + dt*dr_I

  return r_E, r_I


dt = 0.1 # time-step
t = np.arange(0, 1000, dt)
r_E, r_I = Euler_Simple_Linear_System(t, dt)
plot_rErI(t, r_E, r_I)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_cce398e8.py)

*Example output:*

<img alt='Solution hint' align='left' width=416 height=272 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials//static/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_cce398e8_0.png>



### Think! 6: Simple Euler Solution to the Wilson Cowan model

1. Is the simulation biologically plausible?
2. What is the effect of combined excitation and inhibition?


[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_13ffd604.py)



## Section 6.1: Phase Plane Plot and Nullcline
When there are two differential equations describing the interaction of two processes like excitation $r_{E}$ and inhibition $r_{I}$ that are both dependent on each other one way of plotting them is as a function of each other which is called a phase plane plot. The phase plane plot can give insight give insight into the state of the system but more about that later in Neuromatch Academy.

An addition to the phase plane plot are the "nullcline". These lines plot when the rate of change $\frac{d}{dt}$ of the variables is equation to $0$. We saw a variation of this for a single differential equation in the differential equation tutorial.

As we have have two differential equations we set $\frac{dr_E}{dt}=0$ and $\frac{dr_I}{dt}=0$ which gives the equations,

\begin{align}
0&= w_{EI}r_I, \\
0&= w_{IE}r_E,\\
\end{align}

these are a unique example as they are a vertical and horizontal line. Where the lines cross is the stable point which the $r_E(t)$ excitatory population and $r_I(t)$ the inhibitory population orbit around.


In [ ]:
# @markdown Excute the code to plot the solution to the system
plot_rErI_Simple(t, r_E, r_I)

### Think! 6.1: Discuss the Plots

1. Which representation is more intuitive (and useful), the time plot or the phase plane plot?
2. Why do we only see one circle? 
3. What do the quadrants represent?


[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_92736bfb.py)



---
# Section 7: Linear System of Differential Equations

Building up the equations in the previous section we re-introduce internal connections $w_{EE}$, $w_{II}$. The two coupled differential equations, each representing the dynamics of the excitatory or inhibitory population are now:

\begin{align}
\tau_E \frac{dr_E}{dt} &=w_{EE}r_E +w_{EI}r_I, \\
\tau_I \frac{dr_I}{dt} &=w_{IE}r_E +w_{II}r_I ,
\end{align}

where $\tau_E=100$ and $\tau_I=120$, $w_{EE}=1$ and $w_{II}=-1$, and $w_{EI}=-5$ and $w_{IE}=0.6$,
with the initial conditions

\begin{align}
r_E(0)=30, \\ 
r_I(0)=20.
\end{align}

The solution can be approximated using the Euler method such that the equations become:

\begin{align*}
\frac{\color{red}{rE_{i+1}}-\color{green}{rE_i}}{\color{blue}{\Delta t}}&=\big(\frac{\color{blue}{w_{EE}}\color{green}{rE_i}+\color{blue}{w_{EI}}\color{green}{rI_i}}{\color{blue}{\tau_E}}\big),\\
\frac{\color{red}{rI_{i+1}}-\color{green}{rI_i}}{\color{blue}{\Delta t}}&=\big(\frac{\color{blue}{w_{II}}\color{green}{rI_i}+\color{blue}{w_{IE}}\color{green}{rE_i}}{\color{blue}{\tau_I}}\big),\\
\end{align*}

Re-arranging:

\begin{align*}
\color{red}{rE_{i+1}}&=\color{green}{rE_i}+\color{blue}{\Delta t}\big(\frac{\color{blue}{w_{EE}}\color{green}{rE_i}+\color{blue}{w_{EI}}\color{green}{rI_i}}{\color{blue}{\tau_E}}\big),\\
\color{red}{rI_{i+1}}&=\color{green}{rI_i}+\color{blue}{\Delta t}\big(\frac{\color{blue}{w_{II}}\color{green}{rI_i}+\color{blue}{w_{IE}}\color{green}{rE_i}}{\color{blue}{\tau_I}}\big),\\
&\text{ for } i=0, \cdots n-1,\\\\
rE_0&=30,\\
rI_0&=20.\\
\end{align*}

where $\color{red}{rE_{i+1}}$ and  $\color{red}{rI_{i+1}}$ are unknown, $\color{green}{rE_{i}} $ and  $\color{green}{rI_{i}} $ are known, $\color{blue}{w_{EI}}=-1$, $\color{blue}{w_{IE}}=1$ and $\color{blue}{\tau_E}=100$ and $\color{blue}{\tau_I}=120$  are known parameters and $\color{blue}{\Delta t}=0.1$ is a chosen time-step.

## Coding Exercise 7: Implement Euler for a System of Equations
Code the Euler method for the linear system,
\begin{align}
\tau_E \frac{dr_E}{dt} &=w_{EE}r_E +w_{EI}r_I, \\
\tau_I \frac{dr_I}{dt} &=w_{IE}r_E +w_{II}r_I,
\end{align}

where $\tau_E=100$ and $\tau_I=120$, $w_{EE}=1$ and $w_{II}=-1$, and $w_{EI}=-5$ and $w_{IE}=0.6$,
with the initial conditions,
\begin{align}
r_E(0)=30, \\ 
r_I(0)=20.
\end{align}
While this might seem very repititive to an exercise above, it just shows how straightforward it is to convert a system of differential equations into simple pluses and minuses and a bit of multiplication.

In [ ]:
def Euler_Linear_System_Matrix(t, dt, w_EE=1):
  """
  Args:
    time: time
    dt: time-step
    w_EE: Excitation to excitation weight
  Returns:
    r_E: Excitation Firing Rate
    r_I: Inhibition Firing Rate
    N_Er: Nullclines for drE/dt=0
    N_Ir: Nullclines for drI/dt=0
  """

  tau_E = 100
  tau_I = 120
  n = len(t)
  r_I = 20*np.ones(n)
  r_E = 30*np.ones(n)
  w_EI = -5
  w_IE = 0.6
  w_II = -1

  for i in range(n-1):

    ########################################################################
    # TODO for students: compute drE and drI and remove
    raise NotImplementedError("Student exercise: compute drE and drI")
    ########################################################################

    # Calculate the derivatives of the E and I populations
    drE = ...
    r_E[i+1] = r_E[i] + dt*drE

    drI = ...
    r_I[i+1] = r_I[i] + dt*drI

  N_Er = -w_EE / w_EI*r_E
  N_Ir = -w_IE / w_II*r_E

  return r_E, r_I, N_Er, N_Ir

dt = 0.1 # time-step
t = np.arange(0, 1000, dt)
r_E, r_I, _, _ = Euler_Linear_System_Matrix(t, dt)
plot_rErI(t, r_E, r_I)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_e9b2af6f.py)

*Example output:*

<img alt='Solution hint' align='left' width=416 height=272 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials//static/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_e9b2af6f_0.png>



### Think! 7: Oscillations


1. What will happen to the oscillations if the time period is extended? 
2. How would you control or predict the oscillations? 

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials//solutions/tutorials_W0D4_Calculus_W0D4_Tutorial3_Solution_5929a354.py)



## Section 7.1: Phase Plane and Nullclines

Like before, we have two differential equations so we cane plot the results on a phase plane. We can also calculate the Nullclines when we set $\frac{dr_E}{dt}=0$ and $\frac{dr_I}{dt}=0$ which gives,
\begin{align}
0&= w_{EE}r_E+w_{EI}r_I, \\
0&= w_{IE}r_E+w_{II}r_I,
\end{align}
re-arranging as two lines
\begin{align}
r_I&= -\frac{w_{EE}}{w_{EI}}r_E, \\
r_I&= -\frac{w_{IE}}{w_{II}}r_E, 
\end{align}
which crosses at the stable point. 

The panel on the left shows excitation firing rate $r_E$ and inhibition firing rate  $r_I$ as a function of time. On the rightside the phase plane plot shows inhibition firing rate $r_I$ as a function of excitiation firing rate $r_E$ with the Nullclines for $\frac{dr_E}{dt}=0$ and $\frac{dr_I}{dt}=0.$

In [ ]:
# @markdown *Run this cell to visualize the phase plane*
dt = 0.1 # time-step
t = np.arange(0, 1000, dt)
r_E, r_I, N_Er, N_Ir = Euler_Linear_System_Matrix(t, dt)
plot_rErI_Matrix(t, r_E, r_I, N_Er, N_Ir)

### Interactive Demo 7.1: A small change changes everything
To illustrate that even changing one parameter in system of differential equations has a large impact on the solutions of the excitation firing rate $r_E$ and inhibition firing rate $r_I$.
Take note of:
1. How the solution changes for positive and negative of $w_{EE}$ pay close attention to the axis.

2. How would you maintain a stable oscillation.

In [ ]:
# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    w_EE=widgets.FloatSlider(1, min=-1., max=2., step=.1,
                                layout=my_layout)
)

def Pop_widget(w_EE):
  dt = 0.1 # time-step
  t = np.arange(0,1000,dt)
  r_E, r_I, N_Er, N_Ir = Euler_Linear_System_Matrix(t, dt, w_EE)
  plot_rErI_Matrix(t, r_E, r_I, N_Er, N_Ir)
  plt.show()

---
# Take Home






In [ ]:
#@title Video 4: Numerical Methods - Take Home and Bonus Materials
video = YouTubeVideo(id="NDXcNZXeLsw", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

Using pretty much the formula for the slope of a line, the solution of differential equation can be estimated with reasonable accuracy. 
This will be much more relevant when dealing with more complicated (non-linear) differential equations where there is no known exact solution.

---
# Links to Neuromatch Days

The Euler method is used to estimate differential equations in a number of different Neuromatch days:
* The LIF model is discussed in more details in Model Types (Week 1 Day 1) and Real Neurons (Week 2 Day 3).
* Drift Diffusion model which is a differential equation for decision making is discussed in Linear Systems (Week 2 Day 2).
* Phase-plane plots are discussed in Linear Systems (Week 2 Day 2) and Dynamic Networks (Week 2 Day 4).
* The Wilson-Cowan model is discussed in Dynamic Networks (Week 2 Day 4).




---
# References 
1. Lotka, A. L, (1920)   Analytical note on certain rhythmic  relations inorganic  systems.Proceedings of the National Academy of Sciences,6(7):410–415,1920.

2. Brunel N, van Rossum MC. Lapicque's 1907 paper: from frogs to integrate-and-fire. Biol Cybern. 2007 Dec;97(5-6):337-9. doi: 10.1007/s00422-007-0190-0. Epub 2007 Oct 30. PMID: 17968583.



# Bibliography
1. Dayan, P., & Abbott, L. F. (2001). Theoretical neuroscience: computational and mathematical modeling of neural systems. Computational Neuroscience Series.
2. Strogatz, S. Nonlinear dynamics and chaos: with applications to physics, biology, chemistry, and engineering (studies in nonlinearity), Westview Press; 2 edition (29 July 2014)

## Supplemental Popular Reading List
1. Lindsay, G. (2021). Models of the Mind: How Physics, Engineering and Mathematics Have Shaped Our Understanding of the Brain. Bloomsbury Publishing.
2. Strogatz, S. (2004). Sync: The emerging science of spontaneous order. Penguin UK.

## Popular Podcast
1. Strogatz, S. (Host). (2020-), Joy of X  https://www.quantamagazine.org/tag/the-joy-of-x/ Quanta Magazine

---
# Bonus 1: The 4th Order Runge Kutta
Another popular numerical method to estimate the solution of a differential equations of the general form: 
$$ \frac{d}{dt}y=f(t,y)$$
is the 4th Order Runge Kutta: 
$$k_1=f(t,y),$$
$$k_2=f(t+\frac{\Delta t}{2},y+\frac{\Delta t}{2}k_1),$$
$$k_3=f(t+\frac{\Delta t}{2},y+\frac{\Delta t}{2}k_2),$$
$$k_4=f(t+dt,y+\Delta tk_3),$$
$$y_{i+1}=y_{i}+\frac{\Delta t}{6}(k_1+2k_2+2k_3+k_4),$$
which is more accurate than the Euler method.

Given the population differential equation
\begin{align*}
\\
\frac{d}{dt}\,p(t) &= 0.3 p(t)\\
p(t_0=1)&=e^{0.3}\quad \text{Initial Condition}
\end{align*}
the 4th Runge Kutta difference equation is
\begin{align*}
k_1&=0.3\color{green}{p_i},\\
k_2&=0.3(\color{green}{p_i}+\frac{\Delta t}{2}k_1),\\
k_3&=0.3(\color{green}{p_i}+\frac{\Delta t}{2}k_2),\\
k_4&=0.3(\color{green}{p_i}+k_3),\\
\color{red}{p_{i+1}}&=\color{green}{p_i}+\frac{\color{blue}{\Delta t}}{6}( \color{green}{k_1+2k_2+2k_3+k_4})\\
&\text{for } i=0,1,\cdots 4
\end{align*}
to estimate the population for $\Delta t=0.5$.

The code is implemented below. Note how much more accurate the 4th Order Runge Kutta (yellow) is compared to the Euler Method (blue).


In [ ]:
dt=0.5

t=np.arange(1, 5+dt/2, dt)

n = len(t)
p = np.ones(n)
pRK4 = np.ones(n)
p[0] = np.exp(0.3*t[0])
pRK4[0] = np.exp(0.3*t[0])# Initial Condition

with plt.xkcd():

  fig = plt.figure(figsize=(6, 4))
  plt.plot(t, np.exp(0.3*t), 'k', label='Exact Solution')

  for i in range(n-1):
    dp = dt*0.3*p[i]
    p[i+1] = p[i] + dp # Euler
    k1 = 0.3*(pRK4[i])
    k2 = 0.3*(pRK4[i] + dt/2*k1)
    k3 = 0.3*(pRK4[i] + dt/2*k2)
    k4 = 0.3*(pRK4[i] + dt*k3)
    pRK4[i+1] = pRK4[i] + dt/6 *(k1 + 2*k2 + 2*k3 + k4)

  plt.plot(t, p,':o', label='Euler Estimate')
  plt.plot(t, pRK4,':o', color='y', label='4th Order Runge Kutta Estimate')
  plt.vlines(t,p,np.exp(0.3*t),colors='r', linestyles='dashed',label=r'Error $e_i$')

  plt.plot(t[0], p[0], 'go', label='Intial Condition')
  plt.ylabel('Population (millions)')
  plt.legend()
  plt.xlabel('Time (years)')
  plt.show()

### Bonus Reference 1: A full course on numerical methods in Python

For a full course on Numerical Methods you can look here https://github.com/john-s-butler-dit/Numerical-Analysis-Python

# Bonus 2: Neural oscillations are a start toward understanding brain activity rather than the end
In an essay from May 2021 in PLOS Biology by Keith Dowling (a past Neuromatch TA) and M. Florencia Assaneo discussed a mathematical way of thinking about what should be defined as an oscillation.
This is an important question as a good deal of research has investigated oscillations of the brain, and as we saw earlier, a good deal of the equations we use to model neuronal activity result in an oscillation.

They motivated their approach using the Stuart–Landau equations which is a system of differential equations of the form

\begin{align}
 \frac{dx}{dt} &=\lambda x-\omega y -\gamma (x^2+y^2)x+s\\
 \frac{dy}{dt} &=\lambda y+\omega x -\gamma (x^2+y^2)y 
\end{align}

where $s$ is input to the system, and $\lambda$, $\omega$ and $\gamma$ are parameters.
The Stuart–Landau equations are a well described system of differential equations that generates oscillations. For their purpose, they used the equations to illustrate what people should expect when conducting experiments looking for oscilations. 
In their paper, using the Stuart–Landau equations, they outline 
* "What is an oscillator?"
* "What an oscillator is not"
* "Not all that oscillates is oscillator"
* "Not all oscillators are alike."

The Euler form of the Stuart–Landau system of equations is:
\begin{align*}
\color{red}{x_{i+1}}&=\color{green}{x_i}+\color{blue}{\Delta t}\big(\lambda \color{green}{x_i}-\omega \color{green}{y_i} -\gamma (\color{green}{x_i}^2+\color{green}{y_i}^2)\color{green}{x_i}+s\big),\\
\color{red}{y_{i+1}}&=\color{green}{y_i}+\color{blue}{\Delta t}\big(\lambda \color{green}{y_i}+\omega \color{green}{x_i} -\gamma (\color{green}{x_i}^2+\color{green}{y_i}^2)\color{green}{y_i} \big),\\
&\text{ for } i=0, \cdots n-1,\\
x_0&=1,\\
y_0&=1,\\
\end{align*}
with $ \Delta t=0.1/1000$ ms.


In [ ]:
# @title Helper functions
def plot_Stuart_Landa(t, x, y, s):
  """
    Args:
      t  : time
      x  : x
      y  : y
      s : input
    Returns:
      figure with two panels
      top panel: Input as a function of time
      bottom panel: x
  """

  with plt.xkcd():
    fig = plt.figure(figsize=(14, 4))
    gs = gridspec.GridSpec(2, 2,  height_ratios=[1, 4], width_ratios=[4, 1])

    # PLOT OF INPUT
    plt.subplot(gs[0])
    plt.ylabel(r'$s$')
    plt.plot(t, s, 'g')
    #plt.ylim((2,4))

    # PLOT OF ACTIVITY
    plt.subplot(gs[2])
    plt.plot(t ,x)
    plt.ylabel(r'x')
    plt.xlabel(r't')
    plt.subplot(gs[3])
    plt.plot(x,y)
    plt.plot(x[0], y[0],'go')
    plt.xlabel(r'x')
    plt.ylabel(r'y')
    plt.show()


def Euler_Stuart_Landau(s,time,dt,lamba=0.1,gamma=1,k=25):
  """
  Args:
    I: Input
    time: time
    dt: time-step
  """

  n = len(time)
  omega = 4 * 2*np.pi
  x = np.zeros(n)
  y = np.zeros(n)
  x[0] = 1
  y[0] = 1

  for i in range(n-1):
    dx = lamba*x[i] - omega*y[i] - gamma*(x[i]*x[i] + y[i]*y[i])*x[i] + k*s[i]
    x[i+1] = x[i] + dt*dx
    dy = lamba*y[i] + omega*x[i] - gamma*(x[i]*x[i] + y[i]*y[i])*y[i]
    y[i+1] = y[i] + dt*dy

  return x, y

### Bonus 2.1: What is an Oscillator?
Doelling & Assaneo (2021), using the Stuart–Landau system, show different possible states of an oscilator by manipulating the $\lambda$ term in the equation. 
From the paper: "this qualitative change in behavior takes place at λ = 0: For λ < 0, the system decays to a stable equilibrium, while for λ > 0, it keeps oscillating."

#### Interactive Demo Bonus 2.1 
The plot below shows the estimated solution of the Stuart–Landau system with a base frequency $\omega$ set to $4\times 2\pi$, $\gamma=1$ and $k=25$ over 3 seconds. The input to the system $s(t)$ is plotted in the top panel, $x$ as a function of time in the the bottom panel and on the right the phase plane plot of $x$ and $y$. You can manipulate $\lambda$ to see how the oscilations change.

In [ ]:
# @markdown Make sure you execute this cell to enable the widget!
dt=0.1/1000
t=np.arange(0, 3, dt)

my_layout.width = '450px'
@widgets.interact(
    lamda=widgets.FloatSlider(1, min=-1., max=5., step=0.5,
                                layout=my_layout)
)

def Pop_widget(lamda):
    s=np.zeros(len(t))
    x,y=Euler_Stuart_Landau(s,t,dt,lamda)
    plot_Stuart_Landa(t, x, y, s)
    plt.show()

### Bonus 2.2 : Not all oscillators are alike

#### Interactive Demo Bonus 2: Stuart-Landau System 
The plot below shows estimated solution of the Stuart–Landau system with a base frequency $\omega$ set to $4\times 2\pi$, $\lambda=1$, $\gamma=1$ and $k=50$ over 3 seconds the input to the system $s(t)=\sin(freq 2\pi t) $ in the top panel, $x$ as a function of time in the the bottom panel and on the right the phase plane plot of $x$ and $y$. 

You can manipulate the frequency $freq$ of the input to see how the oscilations change and for frequencies $freq$ further and further from the base frequency of 4Hz the oscilations breaks down.

In [ ]:
# @markdown Make sure you execute this cell to enable the widget!
dt=0.1/1000
t=np.arange(0, 3, dt)

my_layout.width = '450px'
@widgets.interact(
    freq=widgets.FloatSlider(4, min=0.5, max=10., step=0.5,
                                layout=my_layout)
)

def Pop_widget(freq):
    s = np.sin(freq * 2*np.pi * t)

    x, y = Euler_Stuart_Landau(s, t, dt, lamba=1, gamma=.1, k=50)
    plot_Stuart_Landa(t, x, y, s)

### Bonus Reference 2
[Doelling, K. B., & Assaneo, M. F. (2021). Neural oscillations are a start toward understanding brain activity rather than the end. PLoS biology, 19(5), e3001234.](https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3001234)
